In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
data_folder, roi = global_variables()

ModuleNotFoundError: No module named 'ee'

## Categorical Variables - Export categorical

Intakes:

    Indigenous land from FUNAI
        https://www.gov.br/funai/pt-br/atuacao/terras-indigenas/geoprocessamento-e-mapas
    Ecoregion from RESOLVE
        https://developers.google.com/earth-engine/datasets/catalog/RESOLVE_ECOREGIONS_2017
    Protected areas from CEM - USP (2020)
        https://centrodametropole.fflch.usp.br/pt-br/download-de-dados
    Soil type from DSMW
        https://data.apps.fao.org/map/catalog/srv/eng/catalog.search#/metadata/446ed430-8383-11db-b9b2-000d939bc5d8
    Biome data from IBGE
        https://www.ibge.gov.br/geociencias/informacoes-ambientais/vegetacao/15842-biomas.html

        For specific biomes, the feature "CD_Bioma" is:
        1 = Amazonia
        2 = Caatinga
        3 = Cerrado
        4 = Mata Atlantica
        5 = Pampa
        6 = Pantanal
    
Exports images with binary masks for protected areas and indigenous land, or byte values for ecoregion and soil type.

In [2]:

"""
fc_address -> GEE path to the feature collection
column_name -> column that designates the value of the feature we want to extract (in this case, the ID)
newname -> band name for the new image
"""

# Soil categories are strings, which Image objects can't handle, and ecoregion categories are large integers.
# To transform the feature collection into an image, we convert the categories to bytes by creating a dictionary.


def categorical_convert(fc_address, column_name, newname):
    # import feature collection and crop it to region of interest
    fc = (
        ee.FeatureCollection(fc_address)
        .filterBounds(roi)
        .map(lambda feature: feature.intersection(roi))
    )
    # create array with unique categories in the feature collection
    unique_category = fc.select([column_name]).aggregate_array(column_name).distinct()
    # assign numbers starting from 1 to each category
    category_dict = ee.Dictionary.fromLists(
        unique_category, ee.List.sequence(1, unique_category.length())
    )
    # reassign original category values to new values from dictionary (1 to total number of categories)
    converted_fc = fc.remap(unique_category, category_dict.values(), column_name)
    # return image with new values in byte type to facilitate exporting
    return (
        converted_fc.reduceToImage([column_name], ee.Reducer.first())
        .byte()
        .rename(newname)
    )

In [3]:
ecoregions = (
    ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
    .filterBounds(roi)
    .map(lambda feature: feature.intersection(roi))
)
ecoregions_img = ecoregions.reduceToImage(["ECO_ID"], ee.Reducer.first()).toInt16().rename(
    "ecoreg"
)

biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
biome_img = biome.reduceToImage(["CD_Bioma"], ee.Reducer.first()).byte().rename("biome")

soil_img = categorical_convert(f"{data_folder}/raw/DSMW_soil", "DOMSOI", "soil").byte()

indig_land = ee.FeatureCollection(f"{data_folder}/raw/indig_land").filterBounds(roi)
indig_land_img = ee.Image().paint(indig_land, 1).unmask(0).byte().rename("indig")

protec = ee.FeatureCollection(f"{data_folder}/raw/protec_2021").filterBounds(roi)
protec_img = ee.Image().paint(protec, 1).unmask(0).byte().rename("protec")

In [4]:
categorical = ecoregions_img.addBands([biome_img, soil_img, indig_land_img, protec_img])

export_image(categorical, "categorical", scale = 500)

if panama:
    export_image(categorical, "categorical", folder = "panama", scale = 500)

In [ ]:
gaul = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0')

south_american_countries = [
    'Argentina', 'Bolivia', 'Chile', 'Colombia', 'Ecuador',
    'French Guiana', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

south_america = gaul.filter(ee.Filter.inList('ADM0_NAME', south_american_countries))
sa = ee.Image.constant(1).clip(south_america).reproject(crs = 'EPSG:4326', scale = 5000)

categorical = ee.Image(f"{data_folder}/categorical")

biome_mask = categorical.select('biome').eq(1) \
.Or(categorical.select('biome').eq(4)) \
.Or(categorical.select('biome').eq(6)).reproject(crs = 'EPSG:4326', scale = 5000)

new_sa = ee.Image(-1).where(sa.mask().Not(), 2) \
                      .where(biome_mask.gt(0), 1) \
                      .where(biome_mask.eq(0), -1)

edge_detec = new_sa.zeroCrossing()

distance_to_border = edge_detec.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).updateMask(biome_mask).rename("distance_to_edge")
distance_to_border_mask = distance_to_border.gt(10000)

export_image(distance_to_border_mask, "distance_to_border_mask", scale = 1000)

In [ ]:
gaul = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0')

central_american_countries = [
    'Colombia', 'Panama', 'Costa Rica'
]

central_america = gaul.filter(ee.Filter.inList('ADM0_NAME', central_american_countries))
ca = ee.Image.constant(1).clip(central_america).reproject(crs = 'EPSG:4326', scale = 5000)

categorical = ee.Image(f"{data_folder}/categorical")

new_ca = ee.Image(-1).where(sa.mask().Not(), 2) \
                      .where(biome_mask.gt(0), 1) \
                      .where(biome_mask.eq(0), -1)

edge_detec = new_ca.zeroCrossing()

distance_to_border = edge_detec.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).updateMask(biome_mask).rename("distance_to_edge")
distance_to_border_mask = distance_to_border.gt(10000)

export_image(distance_to_border_mask, "distance_to_border_mask", folder = "panama", scale = 1000)

export_image(categorical, "categorical", folder = "panama", scale = 500)